In [1]:
import random
random.seed(8266)
import functools

In [ ]:
# We assume that the leading hand is north. If this is not true, the board can be easily rearranged to satisfy such a condition.

hands = [str(index + 1) + suit for index in range(13) for suit in ["C", "D", "H", "S"]]
consumedBids = [False, False, False, False]
generatedHands = [[], [], [], []]

POWER_TO_DENSITY = {
    1: 4,
    2: 4,
    3: 5,
    4: 6,
    5: 7,
    6: 7,
    7: 8
}

In [ ]:
bids: list[list] = [
    ["1D", "1NT", "Pass"],
    ["Pass", "Pass", "Pass"],
    ["1H", "2H", "Pass"],
    ["Pass", "Pass"]
]

# usedCards = [["13S"], ["4D"], ["2H"], ["3H"]]

In [4]:
def evalHandStrength(cards: list[str]):
    points = 0
    weights = {"C": 0, "D": 0, "H": 0, "S": 0}
    for card in cards:
        if int(card[:-1]) > 10: points += int(card[:-1]) - 10
        weights[card[-1]] += 1
    return points, weights

In [5]:
def orderHand(card1, card2):
    if card1[-1] != card2[-1]: 
        if card1[-1] < card2[-1]: return -1
        else: return 1
    else: 
        if card1[:-1] == "1": return 1
        if card2[:-1] == "1": return -1
        return int(card1[:-1]) - int(card2[:-1])

In [6]:
numberConversionTable = {"10": "T", "11": "J", "12": "Q", "13": "K", "1": "A"}
def replaceNumbers(card: str):
    for k, v in numberConversionTable.items():
        if k in card: card = card.replace(k, v)
    return card

In [7]:
def split_fourths(input_list: list[list], replaceCards=False) -> str:
    flattenedList = []
    for i in range(sum([len(x) for x in input_list])): #38 possible bids
        if i // 4 < len(input_list[i % 4]): 
            val = input_list[i % 4][i // 4]
            if replaceCards: val = replaceNumbers(val)
            flattenedList.append(val)
        else: break
    return "\n".join([" ".join(flattenedList[i:i+4]) for i in range(0, len(flattenedList), 4)])
    

In [8]:
# Grabbing the final bid of a section, which determines the contact
finalBids = []
for index, bidSet in enumerate(bids):
    if any(bidSet):
        finalBids.append([x for x in bidSet if x != "Pass"])
        finalBids[index] = finalBids[index][-1]
    else: finalBids[index] = None
finalBid = [x for x in [y[-1] for y in bids] if x != "Pass"][0]
finalBidIndex = finalBids.index(finalBid)


numSuits = []
for index, bid in enumerate(finalBids):
    if bid: consumedBids[index] = True
    else: numSuits[index] = None
    bidPower, bidSuit = int(bid[0]), bid[1]
    totalInSuit = sum([int(b[0]) for b in finalBids if bid[1] == bidSuit])
    samples = POWER_TO_DENSITY[bidPower] 
    samples = max(samples - sum([x for x in generatedHands if bidSuit in x]), 0)
    numSuits.append(round(samples * (bidPower / totalInSuit)))

for index, bidSet in enumerate(bids):
    if len(usedCards[index]) > 0:
        generatedHands[index].extend(usedCards[index])
    for card in usedCards[index]:
        hands.remove(card)
    handSubset = random.sample([x for x in hands if bidSuit in x], numSuits[index])
    generatedHands[index].extend(handSubset)
    for card in handSubset:
        hands.remove(card)


for index, bidSet in enumerate(generatedHands):
    handSubset = random.sample(hands, 13 - len(bidSet))
    generatedHands[index].extend(handSubset)
    
    for card in handSubset:
        hands.remove(card)

sortedHands = []
for hand in generatedHands:
    sortedHand = [x for x in hand]
    sortedHand.sort(key=functools.cmp_to_key(orderHand), reverse=True)
    sortedHands.append(sortedHand)
print([[replaceNumbers(y) for y in x] for x in sortedHands])

[['AS', 'KS', 'TS', '9S', '7S', 'QH', 'TH', '9H', '6H', 'AD', 'AC', 'TC', '9C'], ['QS', '6S', '5S', '4S', 'KD', '9D', '8D', '7D', '4D', '3D', '2D', '7C', '3C'], ['JS', '3S', '2S', 'KH', '8H', '2H', 'QD', '5D', 'KC', 'JC', '6C', '5C', '2C'], ['8S', 'AH', 'JH', '7H', '5H', '4H', '3H', 'JD', 'TD', '6D', 'QC', '8C', '4C']]


In [9]:
def to_pbn():
    deal = f"{leadingHand}:"
    for hand in sortedHands:
        previousSuit = "S"
        for card in hand:
            if previousSuit not in card: deal += "."
            previousSuit = card[-1]
            deal += replaceNumbers(card[:-1])
        deal += " "
    deal = deal.strip()

    pbn = f"""[Event "#"]
    [Site "#"]
    [Date "#"]
    [Round "#"]
    [Board "1"]
    [West "West"]
    [North "North"]
    [East "East"]
    [South "South"]
    [Dealer "{leadingHand}"]
    [Vulnerable "None"]
    [Deal "{deal}"]
    [Declarer "{handOrder[finalBidIndex]}"]
    [Contract "{finalBid}"]
    [Result "#"]
    [HomeTeam "#"]
    [Room "Open"]
    [Scoring "#"]
    [Score "#"]
    [Stage "#"]
    [VisitTeam "#"]
    [Auction "{leadingHand}"]
    {split_fourths(bids)}
    [Play "{handOrder[1]}"]
    {split_fourths(usedCards, replaceCards = True)}
    """

    with open("export.pbn", "w") as f:
        f.write(pbn)

to_pbn()